In [31]:
#importing all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as  sns
import torch as pt
# import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

# from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

#load csv file
file_path = 'C:\\Users\\aurel\\Downloads\\WildGuard\\ml-ai\\comptab_2018-01-29 16_00_comma_separated.csv'
data = pd.read_csv(file_path, delimiter = ',')

In [21]:
#Select relevant columns
selected_columns = [
    'Year', 'App.', 'Family', 'Genus', 'Importer', 'Exporter',
    'Origin', 'Importer reported quantity', 'Exporter reported quantity',
    'Term'
]
data_selected = data[selected_columns]

#Data Cleaning
data_selected.replace('-', np.nan, inplace=True)

#Fill Missing Values
data_selected.fillna(method='ffill', inplace=True)

#Filter the data to keep only rows where 'Origin' is not NaN
data_selected = data_selected.dropna(subset=['Origin'])

C:\Users\aurel\AppData\Local\Temp\ipykernel_14712\2261941329.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected.replace('-', np.nan, inplace=True)
C:\Users\aurel\AppData\Local\Temp\ipykernel_14712\2261941329.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected.fillna(method='ffill', inplace=True)


In [33]:
#Country codes
country_codes = [
    'DZ', 'AO', 'BJ', 'BW', 'BF', 'BI',
    'CV', 'CM', 'CF', 'TD', 'KM',
    'CG', 'CI', 'CD', 'DJ',
    'EG', 'GQ', 'ER', 'SZ', 'ET', 'GA',
    'GM', 'GH', 'GN', 'GW', 'KE', 'LS', 'LR',
    'LY', 'MG', 'MW', 'ML', 'MR', 'MU',
    'MA', 'MZ', 'NA', 'NE', 'NG', 'RW',
    'ST', 'SN', 'SC', 'SL', 'SO',
    'ZA', 'SS', 'SD', 'TZ', 'TG', 'TN',
    'UG', 'ZM', 'ZW'
]

filtered_data = data_selected[data_selected['Origin'].isin(country_codes)]

#Filter data to show only rows where 'Family' contains 'Rhino' or 'Elephant'
filtered_data = filtered_data[
    filtered_data['Family'].str.contains('Rhinoceros|Elephantidae', case=False, na=False)
]

print(filtered_data)

       Year App.        Family      Genus Importer Exporter Origin  \
1705   2016    I  Elephantidae     Dugong       NZ       FR     KE   
1709   2016    I  Elephantidae     Dugong       NZ       HU     AO   
1775   2016    I  Elephantidae     Dugong       US       NL     KE   
1930   2016    I  Elephantidae  Loxodonta       AR       ZA     TZ   
1931   2016    I  Elephantidae  Loxodonta       AR       ZA     TZ   
...     ...  ...           ...        ...      ...      ...    ...   
30060  2016   II  Elephantidae  Loxodonta       ZM       ZW     BW   
30061  2016   II  Elephantidae  Loxodonta       ZM       ZW     BW   
67007  2017    I  Elephantidae  Loxodonta       ZA       MW     ZA   
67008  2017    I  Elephantidae  Loxodonta       ZA       MW     ZA   
67009  2017    I  Elephantidae  Loxodonta       ZA       MW     ZA   

       Importer reported quantity  Exporter reported quantity  \
1705                          1.0                         1.0   
1709                         

In [34]:
#Create custom dataset
class AnimalDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels)

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]